# Cifar-10 test for image classification

In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Resizing, Lambda, Flatten, Dense
import pandas as pd
import numpy as np
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

/Users/sasha/jax-metal/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Download DenseNet121 with Imagenet weights (1000 classes)

In [19]:
dnet = tf.keras.applications.densenet.DenseNet121(include_top=True,
                                                  weights='imagenet',
                                                  input_tensor=None,
                                                  input_shape=None,
                                                  pooling=None,
                                                  classes=1000,
                                                  classifier_activation='softmax'
                                                  )

In [20]:
dnet.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d_2 (ZeroPadd  (None, 230, 230, 3)          0         ['input_4[0][0]']             
 ing2D)                                                                                           
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d_2[0][0]']    
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']

Make all layers untrainable except for the very last convolutional layer

In [21]:
for layer in dnet.layers:
    layer.trainable = False
dnet.layers[-6].trainable  = True

Cifar-10 testing

In [22]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [23]:
y_train_cat = to_categorical(y_train, 1000)
y_test_cat = to_categorical(y_test, 1000)

Lambda layer for preprocessing

In [24]:
def preprocess(x):
    x = tf.image.resize(x,size=(224,224),method='bicubic')
    x = tf.keras.applications.densenet.preprocess_input(x)
    return x

Modify the model

In [25]:
input_shape = (32,32,3)

In [26]:
input_layer = Input(shape=input_shape)
prep = Lambda(preprocess)(input_layer)
out = dnet(prep)
dnet_mod = Model(inputs=input_layer, outputs=out)

In [27]:
dnet_mod.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1_categorical_accuracy')])

Try to fit it on Cifar-10 data and then evaluate (there is no hope this is gonna work ...)

In [28]:
dnet_mod.fit(X_train, y_train_cat)

1563/1563 [==============================] - 3842s 2s/step - loss: 5.9553 - top_1_categorical_accuracy: 0.1825


In [29]:
dnet_mod.evaluate(X_test, y_test_cat)

313/313 [==============================] - 648s 2s/step - loss: 4.8529 - top_1_categorical_accuracy: 0.2640


[4.8528923988342285, 0.2639999985694885]

Try the same with adding a Cerebros "add-on" network

In [30]:
INPUT_SHAPES  = [input_shape]
OUTPUT_SHAPES = [10]

Use 10k-15k random samples from Cifar-10 to speed up the process

In [31]:
num_samples = 15_000
rng = np.random.default_rng()
ind = rng.permutation(X_train.shape[0])[:num_samples]

In [32]:
training_x   = [tf.constant(X_train[ind,:,:,:])]
y_train_cat  = to_categorical(y_train[ind], 10)
train_labels = [tf.constant(y_train_cat)]

In [33]:
dnet = tf.keras.applications.densenet.DenseNet121(include_top=True,
                                                  weights='imagenet',
                                                  input_tensor=None,
                                                  input_shape=None,
                                                  pooling=None,
                                                  classes=1000,
                                                  classifier_activation='softmax'
                                                  )

In [35]:
for layer in dnet.layers:
    layer.trainable = False
dnet.layers[-6].trainable  = True

In [36]:
dnet_io = Model(inputs=dnet.layers[0].input,
                outputs=dnet.layers[-2].output)

In [37]:
input_layer = Input(shape=input_shape)
prep = Lambda(preprocess)(input_layer)
out = Flatten()(dnet_io(prep))
base_mod = Model(inputs=input_layer, outputs=out)

In [39]:
activation = 'relu'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 10  # [1, 100]
batch_size = 20
maximum_levels = 4  # [3,7]
maximum_units_per_level = 7  # [2,10]
maximum_neurons_per_unit = 4  # [2,20]

In [40]:
# Final training task
TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
#
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test_cifar10_densenet'
#
meta_trial_number = 42
#
cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.2,
    direction='maximize',
    metric_to_rank_by="val_top_1_categorical_accuracy",
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation='softmax',
    number_of_architecture_moities_to_try=3,
    number_of_tries_per_architecture_moity=2,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.TopKCategoricalAccuracy(
                k=1, name='top_1_categorical_accuracy')
             ],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number,
    base_models=[base_mod])

In [41]:
%%time
result = cerebros_automl.run_random_search()

SimpleCerebrosRandomSearch.input_shapes: [(32, 32, 3)]
nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'4': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'1': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}]
2
Trying to create level 2
We think level 2's predecessors are: [0, 1]
k is: 3 value is: [{'2': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}]
3
Trying to create level 3
We think level 3's predecessors are: [0, 1, 2]
k is: 4 value is: [{'10': <class 'cerebros.units.units.FinalDenseUnit'>}]
4
Trying to create Final le

inputs
KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'), name='NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_InputLevel_0000000000000000_tr_0_InputUnit_0000000000000000_tr_0_0_inp'")

outputs
KerasTensor(type_spec=TensorSpec(shape=(None, 10), dtype=tf.float32, name=None), name='NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000004_tr_0_FinalDenseUnit_0000000000000004_tr_0_0_dns_/Softmax:0', description="created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000004_tr_0_FinalDenseUnit_0000000000000004_tr_0_0_dns_'")
Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
______________________________________________________________________

INFO:tensorflow:Assets written to: 2023_10_21_16_16_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.470867                    0.841083  0.300446   
1  0.304662                    0.893083  0.295500   
2  0.263270                    0.909167  0.293994   
3  0.226823                    0.919167  0.314155   
4  0.200816                    0.931000  0.332571   
5  0.172115                    0.937583  0.379954   
6  0.150458                    0.946500  0.346395   
7  0.130742                    0.949417  0.366537   
8  0.104036                    0.961333  0.414900   
9  0.092128                    0.967000  0.455699   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.896000             0                0   
1                        0.901667             0                0   
2                        0.904000             0                0   
3                        0.899667             0                0   
4                        0.898000             0   

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000004_tr_0_FinalDenseUnit_0000000000000004_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_3_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_5_dns_')>, <KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_3_dns_')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_2_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'Ne

...

INFO:tensorflow:Assets written to: 2023_10_21_16_16_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.264904                    0.906133  0.334645   
1  0.041018                    0.991750  0.394694   
2  0.027192                    0.994083  0.438355   
3  0.016322                    0.996333  0.475004   
4  0.022072                    0.993167  0.578026   
5  0.023345                    0.991833  0.596374   
6  0.025543                    0.991500  0.610430   
7  0.022781                    0.992417  0.635906   
8  0.022238                    0.991750  0.659581   
9  0.023292                    0.991500  0.678422   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.884000             0                0   
1                        0.884000             0                0   
2                        0.883000             0                0   
3                        0.888000             0                0   
4                        0.878333             0   

{'1': <class 'cerebros.units.units.DenseUnit'>}
debug: meta_level_number
Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'model_3')>, <KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'model_3')>, <KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'model_3')>, <KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'model_3')>]
materialized_predecessor_units [<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'model_3')>, <KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'model_3')>, <KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'model_3')>, <KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'model_3')>]
{'10': <class 'cerebros.units.units.FinalDenseUnit'>}
debug: meta_leve

INFO:tensorflow:Assets written to: 2023_10_21_16_16_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000001/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  0.252051                    0.912933  0.342295   
1  0.034482                    0.994000  0.388051   
2  0.019183                    0.996167  0.428428   
3  0.013406                    0.997500  0.485034   
4  0.018032                    0.994333  0.517872   
5  0.019061                    0.994333  0.545307   
6  0.022755                    0.992417  0.570794   
7  0.018599                    0.993667  0.662809   
8  0.014659                    0.995000  0.635707   
9  0.019295                    0.993417  0.709606   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.889000             0                1   
1                        0.884333             0                1   
2                        0.880333             0                1   
3                        0.880333             0                1   
4                        0.880667             0   

In [75]:
print(f'Best accuracy achieved is {result}')
print(f'top-1 categorical accuracy')

Best accuracy achieved is 0.9039999842643738
top-1 categorical accuracy
CPU times: user 147 µs, sys: 301 µs, total: 448 µs
Wall time: 2.49 ms


Evaluating the best model found

In [68]:
best_model_found = cerebros_automl.get_best_model()

In [69]:
#
eval_loss = tf.keras.losses.CategoricalCrossentropy()
#
eval_metrics =\
[tf.keras.metrics.TopKCategoricalAccuracy(k=1,\
            name='eval_top_1_categorical_accuracy'),
 tf.keras.metrics.TopKCategoricalAccuracy(k=5,\
            name='eval_top_5_categorical_accuracy')
]

In [71]:
best_model_found.compile(loss=eval_loss, metrics=eval_metrics)
best_model_found.summary()

Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 NeuralNetworkFuture_000000  [(None, 32, 32, 3)]          0         []                            
 0000000nan_tr_0_InputLevel                                                                       
 _0000000000000000_tr_0_Inp                                                                       
 utUnit_0000000000000000_tr                                                                       
 _0_0_inp (InputLayer)                                                                            
                                                                                                  
 model_3 (Functional)        (None, 1024)                 7037504   ['NeuralNetworkFuture_00000000
                                          

In [76]:
print("Evaluating best model found ...")
print("Loss | Top-1 accuracy | Top-5 accuracy")
y_test_cat = to_categorical(y_test, 10)
best_model_found.evaluate(X_test, y_test_cat)

Evaluating best model found ...
Loss | Top-1 accuracy | Top-5 accuracy
313/313 [==============================] - 552s 2s/step - loss: 0.7619 - eval_top_1_categorical_accuracy: 0.8688 - eval_top_5_categorical_accuracy: 0.9933


[0.761899471282959, 0.8687999844551086, 0.9933000206947327]